На основе word2vec/fasttext реализовать метод поиска ближайших статей (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)

In [7]:
import pandas as pd
import numpy as np
import gensim.downloader as api
import string
import annoy
import re
import os
import gensim.models
from tqdm import tqdm
from tqdm import tqdm_notebook
from corus import load_lenta
from gensim.models import Word2Vec, FastText
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from tensorflow.keras.layers import Dense, Embedding, Flatten

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [8]:
path = '/media/dmitriy/Disk/Downloads/ai_nlp_hw_data/hw_3/lenta-ru-news.csv.gz'
records = load_lenta(path)

In [9]:
corpus_title = []
corpus_text = []

for num, iter in tqdm(enumerate(records)):
    corpus_title.append(iter.title)
    corpus_text.append(iter.text)
    if num == 99999:
        break

0it [00:00, ?it/s]

99999it [00:04, 24214.57it/s]


In [10]:
df = pd.DataFrame(corpus_title, columns=['title'])
df['text'] = corpus_text

In [11]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [12]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [13]:
pattern = r'[^а-яА-Я0-9]'
df['text'] = df['text'].apply(lambda x: re.sub(pattern, ' ', x))

In [14]:
sentences = []
counter = 0

for line in tqdm_notebook(df.text):
    spls = preprocess_txt(line)
    sentences.append(spls)
    counter += 1
    if counter > 100000:
        break

  0%|          | 0/100000 [00:00<?, ?it/s]

In [15]:
modelW2V = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=2)

In [16]:
modelFT = FastText(sentences=sentences, vector_size=100, min_count=2, window=5, workers=8)

In [17]:
w2v_index = annoy.AnnoyIndex(100 ,'angular')
ft_index = annoy.AnnoyIndex(100 ,'angular')
index_map = {}
counter = 0
for line in tqdm_notebook(df.text):
        n_w2v = 0
        n_ft = 0
        spls = line
        index_map[counter] = spls
        question = preprocess_txt(spls)
        vector_w2v = np.zeros(100)
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
        counter += 1
        # ограничение в  100000 строк
        if counter > 100000:
            break
w2v_index.build(10)
ft_index.build(10)

  0%|          | 0/100000 [00:00<?, ?it/s]

True

In [18]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(100)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    return [index_map[i] for i in answers]

In [19]:
TEXT = 'экономика россии'

In [20]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['Официальный представитель Международного валютного фонда  МВФ  Джерри Прайс похвалил российские власти за быстрый выход из кризиса  сообщает ТАСС  По его словам  этому поспособствовала хорошая макроэкономическая политика  Однако российской экономике не хватает структурных реформ  особенно в части улучшения инвестиционного климата  отмечает Райс   Благодаря хорошей макроэкономической политике  повышенному  экономическому  настрою  частично из за более высоких цен на нефть  темпы роста ВВП снова будут положительными     заявил представитель МВФ  Райс советует российским властям наращивать инвестиции в инфраструктуру  социально экономическую сферу  увеличивать эффективность товарного рынка  а также развивать внешнеэкономическое взаимодействие с другими странами  Джерри Райс не первый представитель МВФ  который отмечает успех российских властей в борьбе с кризисом  На Гайдаровском форуме 17 января экономику России также похвалил глава представительства МВФ в России Габриэль Ди Белла  По 

In [21]:
get_response(TEXT, ft_index, modelFT, index_map)

['США стремятся к обострению внешнеполитической обстановки для того  чтобы ослабить Россию  Однако Европе это невыгодно  считает бывший канцлер ФРГ Герхард Шредер  в настоящее время занимающий пост главы совета директоров  Роснефти   Об этом он заявил в четверг  19 октября  на Евразийском форуме  передает  Интерфакс    Европе и  в первую очередь  Германии нужен противоположный результат   процветающая Россия     отметил Шредер  Поэтому  по его словам  прекращение антироссийских санкций будет соответствовать интересам ЕС  Шредер также назвал бессмысленными с политической точки зрения и опасными с экономической попытки США изолировать Россию   Когда в политических отношениях происходит разрыв  невозможно предугадать последствия  В подобных ситуациях мир экономики должен обеспечивать сохранение диалога между сторонами  и мир политики должен к этому прислушиваться     подчеркнул он  Давление США с целью ослабить сотрудничество с европейцами ощущается  по словам Шредера  и в энергетической 